In [1]:
import numpy as np
import logging
from mlp.data_providers import MNISTDataProvider
import matplotlib.pyplot as plt
import pickle

from mlp.layers import AffineLayer, SoftmaxLayer, SigmoidLayer, ReluLayer, LeakyReluLayer, ELULayer, SELULayer
from mlp.errors import CrossEntropySoftmaxError
from mlp.models import MultipleLayerModel
from mlp.initialisers import ConstantInit, GlorotUniformInit, SELUInit
from mlp.learning_rules import GradientDescentLearningRule
from mlp.optimisers import Optimiser

# %matplotlib notebook
plt.style.use('ggplot')
seed = 10102016 

# Coursework 1

This notebook is intended to be used as a starting point for your experiments. The instructions can be found in the instructions file located under spec/coursework1.pdf. The methods provided here are just helper functions. If you want more complex graphs such as side by side comparisons of different experiments you should learn more about matplotlib and implement them. Before each experiment remember to re-initialize neural network weights and reset the data providers so you get a properly initialized experiment. For each experiment try to keep most hyperparameters the same except the one under investigation so you can understand what the effects of each are.

In [2]:
def train_model_and_plot_stats(
        model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True):
    
    # As well as monitoring the error over training also monitor classification
    # accuracy i.e. proportion of most-probable predicted classes being equal to targets
    data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

    # Use the created objects to initialise a new Optimiser instance.
    optimiser = Optimiser(
        model, error, learning_rule, train_data, valid_data, data_monitors, notebook=notebook)

    # Run the optimiser for 5 epochs (full passes through the training set)
    # printing statistics every epoch.
    stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)

    # Plot the change in the validation and training set error over training.
    fig_1 = plt.figure(figsize=(8, 4));
    ax_1 = fig_1.add_subplot(111);
    for k in ['error(train)', 'error(valid)']:
        ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k);
    ax_1.legend(loc=0);
    ax_1.set_xlabel('Epoch number');

    # Plot the change in the validation and training set accuracy over training.
    fig_2 = plt.figure(figsize=(8, 4));
    ax_2 = fig_2.add_subplot(111);
    for k in ['acc(train)', 'acc(valid)']:
        ax_2.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k);
    ax_2.legend(loc=0);
    ax_2.set_xlabel('Epoch number');
    
    return stats, keys, run_time, fig_1, ax_1, fig_2, ax_2

def make_Model(Nl, number_layers):
    layers_and_activation = ([AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), 
    Nl,] * (number_layers-1))
    return MultipleLayerModel([AffineLayer(input_dim, hidden_dim, weights_init, biases_init)] + 
        layers_and_activation + [AffineLayer(hidden_dim, output_dim, weights_init, biases_init)])

In [3]:
# The below code will set up the data providers, random number
# generator and logger objects needed for training runs. As
# loading the data from file take a little while you generally
# will probably not want to reload the data providers on
# every training run. If you wish to reset their state you
# should instead use the .reset() method of the data providers.


# Seed a random number generator
rng = np.random.RandomState(seed)
batch_size = 50
# Set up a logger object to print info about the training run to stdout
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = [logging.StreamHandler()]

# Create data provider objects for the MNIST data set
# train_data = MNISTDataProvider('train', batch_size=batch_size, rng=rng)
# valid_data = MNISTDataProvider('valid', batch_size=batch_size, rng=rng)

In [1]:
#setup hyperparameters
learning_rate = 0.1
num_epochs = 100
stats_interval = 2
input_dim, output_dim, hidden_dim = 784, 10, 100
num_layers = 2

In [7]:
# Setup file for pickle
file_name = 'hiddenLayers'
size = 3 * 6 
fileObject = open(file_name, 'wb')

nonlinearities = [SigmoidLayer(),LeakyReluLayer(),SELULayer()]
for nl in nonlinearities:
    for i in range(2,9):
        rng = np.random.RandomState(seed)
        train_data = MNISTDataProvider('train', batch_size=batch_size, rng=rng)
        valid_data = MNISTDataProvider('valid', batch_size=batch_size, rng=rng)

        # setup weights, biases error and learning
        weights_init = GlorotUniformInit(rng=rng)
        biases_init = ConstantInit(0.)
        error = CrossEntropySoftmaxError()
        # Use a basic gradient descent learning rule
        learning_rule = GradientDescentLearningRule(learning_rate=learning_rate)
        # make model
        model = make_Model(nl, i)

        #Train data
        print("Training {} model with {} hidden layers and {} weight init".format(nl,i,'uniform'))
        model_type = (nl, i,'uniform')
        a = train_model_and_plot_stats(
            model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True);
        pickle.dump((model_type,a), fileObject);

fileObject.close();

Training SigmoidLayer model with 2 hidden layers and uniform weight init


HBox(children=(IntProgress(value=0), HTML(value='')))

Epoch 100: 71.3s to complete
    error(train)=2.19e-02, acc(train)=9.97e-01, error(valid)=8.58e-02, acc(valid)=9.76e-01, params_penalty=0.00e+00


/home/miro/.conda/envs/mlp/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Training SigmoidLayer model with 3 hidden layers and uniform weight init


...

HBox(children=(IntProgress(value=0), HTML(value='')))

Epoch 100: 216.9s to complete
    error(train)=nan, acc(train)=9.86e-02, error(valid)=nan, acc(valid)=9.91e-02, params_penalty=0.00e+00



Training SELULayer model with 8 hidden layers and uniform weight init


HBox(children=(IntProgress(value=0), HTML(value='')))

Epoch 100: 245.2s to complete
    error(train)=nan, acc(train)=9.86e-02, error(valid)=nan, acc(valid)=9.91e-02, params_penalty=0.00e+00
